In [ ]:
import requests
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd
from google.oauth2 import service_account
from google.cloud  import bigquery
from tqdm import tqdm
from pandas import concat
from google.cloud.bigquery import Client
from google.oauth2 import service_account
# libaries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import json
import datetime
import time
import pprint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path  = "/content/drive/MyDrive/AI/dtwo-ec5ea-firebase-adminsdk-ts7ga-8ff16486b9.json"
option_prod = {'projectId': 'dtwo-ec5ea'}
#cred2 = credentials.Certificate("/mnt/c/Users/jabsr/Sources/dtwo-ec5ea-firebase-adminsdk-ts7ga-8ff16486b9.json")
cred2 = credentials.Certificate(path)
prod_app = firebase_admin.initialize_app(cred2, option_prod, 'dtwo-qa')
dtwo_app_db = firestore.client(prod_app)

In [ ]:
from google.api_core.retry import Retry

def getFullCollection(collection):
    model_question_reference = dtwo_app_db.collection(collection)
    docs = model_question_reference.stream(retry=Retry())
    question_list = []
    for doc in docs:
        dictionary = doc.to_dict()
        dictionary["docId"] = doc.id
        question_list.append(dictionary)
    dataframe = pd.DataFrame(question_list)
    return dataframe

In [ ]:
def get_conversations_frame(conversationId):
  document_ref = dtwo_app_db.collection('conversations').document(conversationId)
  subcollection_ref = document_ref.collection('messages')
  docs = subcollection_ref.stream()
  interloc_list = []
  text_list = []
  created_date_list = []
  # Imprimir los documentos de la subcolección
  for doc in docs:
      dictionary_content = doc.to_dict()
      #print(dictionary_content.keys())
      if "text" in dictionary_content.keys() and  "from" in dictionary_content.keys():
        created_date_list.append(dictionary_content["statusHistory"]["created"])
        interloc_list.append(dictionary_content["from"]["type"])
        text_list.append(dictionary_content["text"])

      #print(f'{doc.id} => {doc.to_dict()["text"]}')
      #print(doc.to_dict()["from"])

  comv_frame = pd.DataFrame({"userType": interloc_list, "text":text_list, "createdOn":created_date_list})
  return comv_frame

In [ ]:
def conversations_by_whatsapp_id(whatsappId):
      conversation_collection = dtwo_app_db.collection('conversations')
      print(whatsappId)
      subcollection_ref = conversation_collection.where("whatsappId", "==", whatsappId)
      results = subcollection_ref.get()
      conversationId = ""
      for doc in results:
       conversationId = doc.to_dict()["conversationId"]
      conv_frame = get_conversations_frame(conversationId)
      conv_frame['createdOn'] = pd.to_datetime(conv_frame['createdOn'])
      conv_frame_sorted = conv_frame.sort_values(by='createdOn')

      return conv_frame_sorted

In [ ]:
conv_frame = conversations_by_whatsapp_id("5216631001086")

5216631001086


/usr/local/lib/python3.10/dist-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


In [ ]:
conversation_frame = getFullCollection("conversations")

In [ ]:
conversation_frame

,lastMessageReceived,whatsappId,conversationId,docId
0,2024-04-26 08:04:06+00:00,240222273197,0222273197,0222273197
1,2023-05-21 15:59:03.707000+00:00,50233325008,646a3faa581a20dc3be935e3,0233325008
2,2024-05-18 12:27:24+00:00,50241341364,0241341364,0241341364
3,2023-11-08 12:28:32.164000+00:00,50246330664,625068c6ef14fbdf98feba9f,0246330664
4,2024-01-24 03:41:51.022000+00:00,50247113528,65b086c6424ce7a60a72004d,0247113528
...,...,...,...,...
11051,2023-10-26 15:47:40.039000+00:00,5219999479116,64cbfc3c7d41739149813e39,9999479116
11052,2024-07-06 17:46:30+00:00,5219999479732,9999479732,9999479732
11053,2023-08-17 01:23:15.560000+00:00,5219999551722,64821e78f7719179ae1311c3,9999551722
11054,2024-06-25 17:43:04+00:00,5219999701129,9999701129,9999701129


In [ ]:
import datetime
from datetime import timedelta

In [ ]:
conv_frame["createdOn"] = pd.to_datetime(conv_frame["createdOn"])

In [ ]:
conv_frame["createdOn"].dt.date

457    2024-05-22
191    2024-05-22
83     2024-05-22
271    2024-05-22
339    2024-05-22
          ...    
294    2024-07-06
426    2024-07-06
88     2024-07-06
445    2024-07-06
132    2024-07-06
Name: createdOn, Length: 496, dtype: object

In [ ]:
conv_frame["createdOn"].max().date()

datetime.date(2024, 7, 6)

In [ ]:
conv_frame[(conv_frame["createdOn"].dt.date >= (conv_frame["createdOn"].max().date() -timedelta(days=1)))]

,userType,text,createdOn
140,BOT,Buenos días ☀️. Hoy debemos medirnos la glucos...,2024-07-05 13:30:17.163000+00:00
325,USER,Adelante - glucosa,2024-07-05 14:23:11+00:00
267,BOT,¿Te mediste antes o después de los alimentos? ...,2024-07-05 14:23:13.403000+00:00
410,USER,Antes,2024-07-05 14:23:16+00:00
277,BOT,¿Cuál es tu valor de glucosa? (mg/dL) solo en ...,2024-07-05 14:23:17.581000+00:00
320,USER,150,2024-07-05 14:23:23+00:00
30,BOT,"Buen esfuerzo, estas muy cerca de la meta.",2024-07-05 14:23:25.038000+00:00
155,BOT,¡Mantente activo! 🏃‍♀️🏃‍♂️ Te compartimos una ...,2024-07-05 14:23:25.169000+00:00
153,BOT,"Hola 👋🏼, hoy debemos medirnos la glucosa 🩸 2 h...",2024-07-05 21:30:13.840000+00:00
473,USER,Adelante - glucosa,2024-07-06 15:57:02+00:00


### PRACTICANTE

In [ ]:
conv_frame_text_per_user = conv_frame[(conv_frame["userType"] == "USER")]

In [ ]:
conv_frame_text_per_user

,userType,text,createdOn
457,USER,"Buenos dias, mi nombre es Alejandro Acuna, ten...",2024-05-22 14:04:21+00:00
339,USER,Ya soy paciente,2024-05-22 14:04:45+00:00
400,USER,"Si, lo cambié",2024-05-22 14:04:51+00:00
334,USER,Este 4491554095,2024-05-22 14:05:15+00:00
459,USER,No tengo el link para la consulta,2024-05-22 14:05:52+00:00
...,...,...,...
410,USER,Antes,2024-07-05 14:23:16+00:00
320,USER,150,2024-07-05 14:23:23+00:00
473,USER,Adelante - glucosa,2024-07-06 15:57:02+00:00
426,USER,Antes,2024-07-06 15:57:07+00:00


In [ ]:
conv_frame_for_sentences = conv_frame_text_per_user["text"]
conv_frame_for_sentences

457    Buenos dias, mi nombre es Alejandro Acuna, ten...
339                                      Ya soy paciente
400                                        Si, lo cambié
334                                      Este 4491554095
459                    No tengo el link para la consulta
                             ...                        
410                                                Antes
320                                                  150
473                                   Adelante - glucosa
426                                                Antes
445                                                  133
Name: text, Length: 192, dtype: object

In [ ]:
list_of_sentences = []
for i, row in conv_frame_text_per_user.iterrows():
  list_of_sentences.append(row["text"])
print(type(list_of_sentences))
list_of_sentences
#print(type(conv_frame_text_per_user))

<class 'list'>


['Buenos dias, mi nombre es Alejandro Acuna, tengo consulta hoy a las 11.00 con el medico especialista en diabetes,  radico en Tijuana,',
 'Ya soy paciente',
 'Si, lo cambié',
 'Este 4491554095',
 'No tengo el link para la consulta',
 'Se resolvió',
 'si, gracias estoy en espera',
 'estoy en espera, estoy listo',
 'Cotizar mis medicamentos',
 'Cotizar toda receta',
 'Calificar cita',
 'Endocrinología',
 'Muy satisfecho',
 'Mejor después',
 'Aprovechar oferta',
 'CLIVI - Peso',
 '84',
 'Adelante - glucosa',
 'Antes',
 '126',
 'hola, estoy en espera?',
 'de mi consulta con nutricion',
 'estoy en espera para la consulta',
 'no quiero cancelar',
 'estoy en espera',
 'si, gracias',
 'gracias',
 'Gracias',
 'Calificar cita',
 'Nutricional',
 'Muy satisfecho',
 'Buen dia',
 'Mediciones',
 'Enviar medición',
 'Glucosa',
 'Antes',
 '89',
 'Adelante - glucosa',
 'Antes',
 '169',
 'Medicion',
 'Glucosa',
 'Antes',
 '136',
 'ok',
 'estoy en espera',
 'Calificar cita',
 'Psicológica',
 'Muy satisfe

In [ ]:
pip install tensorflow tensorflow-hub tensorflow-text

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.9 MB/s eta 0:00:00
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.15.1
    Uninstalling tf_keras-2.15.1:
      Successfully uninstalled tf_keras-2.15.1


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embed = hub.load(model_url)

embeddings = embed(list_of_sentences)

In [ ]:
embeddings_np = embeddings.numpy()

In [ ]:
for i, embedding in enumerate(embeddings_np):
    print(f"Oración: {list_of_sentences[i]}")
    print(f"Embedding: {embedding[:10]}...")  # Mostrar solo los primeros 10 valores para mayor claridad
    print(f"Tamaño del vector: {len(embedding)}")
    print()

Oración: Buenos dias, mi nombre es Alejandro Acuna, tengo consulta hoy a las 11.00 con el medico especialista en diabetes,  radico en Tijuana,
Embedding: [ 0.00647483 -0.04288135 -0.0185301   0.05210157 -0.03346945 -0.03316627
  0.03679382  0.02469805 -0.04593856 -0.0076995 ]...
Tamaño del vector: 512

Oración: Ya soy paciente
Embedding: [ 0.02465187 -0.05356941 -0.01654057  0.02031255  0.01783262 -0.04149031
  0.00043687 -0.00749326 -0.01375729  0.00880564]...
Tamaño del vector: 512

Oración: Si, lo cambié
Embedding: [ 0.01476635 -0.05566278  0.01242758  0.04037593  0.0619891   0.04517312
  0.02201864  0.00709474  0.06762496  0.04326826]...
Tamaño del vector: 512

Oración: Este 4491554095
Embedding: [ 0.01070558  0.01273545 -0.01107971  0.04931838  0.03632101 -0.06600614
 -0.02506187 -0.00969726  0.00415375  0.02115691]...
Tamaño del vector: 512

Oración: No tengo el link para la consulta
Embedding: [ 0.04785229  0.0083184  -0.0096941   0.020734   -0.04048952  0.0318607
  0.02860343  

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similaridades = cosine_similarity(embeddings_np)
print("Matriz de similitudes:")
print(similaridades)

Matriz de similitudes:
[[ 0.9999999   0.3014593   0.20616102 ...  0.4713828   0.22800359
   0.02728634]
 [ 0.3014593   1.0000005   0.33434924 ...  0.36326885  0.32868814
   0.07653174]
 [ 0.20616102  0.33434924  1.         ...  0.38493654  0.34617355
  -0.0437742 ]
 ...
 [ 0.4713828   0.36326885  0.38493654 ...  1.          0.3909102
   0.1391304 ]
 [ 0.22800359  0.32868814  0.34617355 ...  0.3909102   1.0000001
   0.16722807]
 [ 0.02728634  0.07653174 -0.0437742  ...  0.1391304   0.16722807
   1.0000002 ]]


In [ ]:
import tensorflow_text as text
from sklearn.cluster import KMeans

num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(embeddings_np)

cluster_labels = kmeans.labels_

# Mostrar los resultados
for i, sentence in enumerate(list_of_sentences):
    print(f"Frase: '{sentence}' está en el cluster {cluster_labels[i]}")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Frase: 'Buenos dias, mi nombre es Alejandro Acuna, tengo consulta hoy a las 11.00 con el medico especialista en diabetes,  radico en Tijuana,' está en el cluster 6
Frase: 'Ya soy paciente' está en el cluster 6
Frase: 'Si, lo cambié' está en el cluster 9
Frase: 'Este 4491554095' está en el cluster 6
Frase: 'No tengo el link para la consulta' está en el cluster 6
Frase: 'Se resolvió' está en el cluster 9
Frase: 'si, gracias estoy en espera' está en el cluster 6
Frase: 'estoy en espera, estoy listo' está en el cluster 6
Frase: 'Cotizar mis medicamentos' está en el cluster 6
Frase: 'Cotizar toda receta' está en el cluster 6
Frase: 'Calificar cita' está en el cluster 8
Frase: 'Endocrinología' está en el cluster 9
Frase: 'Muy satisfecho' está en el cluster 9
Frase: 'Mejor después' está en el cluster 6
Frase: 'Aprovechar oferta' está en el cluster 9
Frase: 'CLIVI - Peso' está en el cluster 9
Frase: '84' está en el cluster 2
Frase: 'Adelante - glucosa' está en el cluster 3
Frase: 'Antes' está 

### FIN DE LA PRACTICA

In [ ]:
def two_day_history(conv_frame):
  conv_frame = conv_frame[(conv_frame["createdOn"].dt.date >= (conv_frame["createdOn"].max().date() -timedelta(days=1)))]
  hitory_conv = ""
  for i,c in conv_frame.iterrows():
    msg = "{} : {}: {} \n \n".format(c["userType"], c["text"], c["createdOn"])
    hitory_conv = hitory_conv + msg
    #print(c["text"])
  return hitory_conv

In [ ]:
extraced_menu = "{}\n\n{}".format("HISTORIAL DE CONVERSACION DEL USUARIO PARA CONTEXTUALIZAR:", two_day_history(conv_frame))


In [ ]:
print(extraced_menu)

HISTORIAL DE CONVERSACION DEL USUARIO PARA CONTEXTUALIZAR:

OPERATOR : _Estimad@ paciente:_
_Nos comunicamos del área de psicología para informarte que desafortunadamente la psicóloga Ixchel ya no labora con nosotros por motivos personales. _
_Por este motivo por vemos en la necesidad de reasignarle un nuevo especialista. _

_Lamentamos el inconveniente y agradecemos su comprensión_
: 2024-07-01 03:41:45.834000+00:00 
 
OPERATOR : 🚨 ✉️ Tu *Psicólog@* *CLIVI* te ha enviado un mensaje 💬. 
Por favor presiona el botón para recibirlo👇

[Recibir mensaje] 

: 2024-07-01 03:41:47.917000+00:00 
 
BOT : 🚨 Tu cita de Psicología ha sido cancelada para el lunes 1 de julio 10:00 a. m.. 🛑

[Agendar nueva cita] 

: 2024-07-01 03:41:56.258000+00:00 
 
USER : Recibir mensaje: 2024-07-01 03:41:58+00:00 
 
BOT : Sin salir de tu casa recibe tus medicamentos

¿Quieres cotizar tus medicamentos y con la posibilidad de envío hasta tu domicilio?💊

Da un toque al botón de abajo *Cotizar mis medicamentos*👇

[Coti

In [ ]:
def extract_menus(conv_frame):
  pattern = r'\['

  conv_frame_menus = conv_frame[conv_frame["text"].str.contains(pattern)]
  conv_frame_menus_cleaned = conv_frame_menus.drop_duplicates(subset='text')

  menu_str = ""

  menu_extracted_list = [c["text"][c["text"].find("["):len(c["text"])] for i,c in conv_frame_menus_cleaned.iterrows()]
  frame_menus = pd.DataFrame({"menus": menu_extracted_list})
  frame_menus_cleaned = frame_menus.drop_duplicates(subset='menus')

  #concatenar cada elemento resultate en un string
  menu_str = ''.join([c["menus"] for i, c in frame_menus_cleaned.iterrows()])

  #fraws_menus = frame_menus.drop_duplicates(subset='menus')

  return menu_str

In [ ]:
extracted_menus = extract_menus(conv_frame)
extracted_menus = extracted_menus.replace("\n", "")

In [ ]:
extraced_menu = "{}\n\n{}".format("MENUS DEL BOT:", extracted_menus)
print(extraced_menu)

MENUS DEL BOT:

[Clivi] [Entrar a la sesión] [Preguntas y estudios][Citas][Mediciones][Entrar a entrenar] [Adelante - glucosa] [Confirmar] [Cancelar] [CLIVI - Peso] [Ver recetario] [Cancelar] [Antes][Después][Ir a cuestionario] [Entrar a la cita] [No asistí a la cita] [Calificar cita] [Psicológica][Nutricional][Endocrinología][🙂🙂🙂][😐😐😐][🙁🙁🙁][Medidas] [Reagendar psicología] [Ver video] [Tratamiento CLIVI] [Requiero apoyo] [En camino] [Recibir mensaje] [Agendar ahora] [Entrar a sesión] [Proceso Nutrición] [Nunca][Ocasionalmente][Siempre][Agendar cita médica] [Quiero aprovechar] [Más adelante] [Enviar pregunta][Labs/Recetas/Planes][Marcar a Clivi][Reenviar receta/labs][Reenviar orden labs][Enviar estudios][Reenviar receta - Última receta][Reenviar orden labs - Última orden de laboratorios][Reenviar estudios - Resultados de estudios][Reenviar plan nutrición - Último plan nutricional][Se resolvió] [Aún no] [Tengo otra duda diferente] [Reagendar nutrición] [Proceso Psicología] [No][Estoy en 

In [ ]:
#sdf = extracted_menu
#print(sdf)

MENUS DEL BOT:[Clivi] [Entrar a la sesión] [Preguntas y estudios][Citas][Mediciones][Entrar a entrenar] [Adelante - glucosa] [Confirmar] [Cancelar] [CLIVI - Peso] [Ver recetario] [Cancelar] [Antes][Después][Ir a cuestionario] [Entrar a la cita] [No asistí a la cita] [Calificar cita] [Psicológica][Nutricional][Endocrinología][🙂🙂🙂][😐😐😐][🙁🙁🙁][Medidas] [Reagendar psicología] [Ver video] [Tratamiento CLIVI] [Requiero apoyo] [En camino] [Recibir mensaje] [Agendar ahora] [Entrar a sesión] [Proceso Nutrición] [Nunca][Ocasionalmente][Siempre][Agendar cita médica] [Quiero aprovechar] [Más adelante] [Enviar pregunta][Labs/Recetas/Planes][Marcar a Clivi][Reenviar receta/labs][Reenviar orden labs][Enviar estudios][Reenviar receta - Última receta][Reenviar orden labs - Última orden de laboratorios][Reenviar estudios - Resultados de estudios][Reenviar plan nutrición - Último plan nutricional][Se resolvió] [Aún no] [Tengo otra duda diferente] [Reagendar nutrición] [Proceso Psicología] [No][Estoy en pr

In [ ]:
print(extracted_menu)

MENUS DEL BOT:

[Clivi] 

[Entrar a la sesión] 

[Preguntas y estudios]
[Citas]
[Mediciones][Entrar a entrenar] 

[Adelante - glucosa] 

[Confirmar] [Cancelar] 

[CLIVI - Peso] 

[Ver recetario] 

[Cancelar] 

[Antes]
[Después][Ir a cuestionario] 

[Entrar a la cita] 

[No asistí a la cita] [Calificar cita] 

[Psicológica]
[Nutricional]
[Endocrinología][🙂🙂🙂]
[😐😐😐]
[🙁🙁🙁][Medidas] 

[Reagendar psicología] 

[Ver video] 

[Tratamiento CLIVI] 

[Requiero apoyo] [En camino] 

[Recibir mensaje] 

[Agendar ahora] 

[Entrar a sesión] 

[Proceso Nutrición] 

[Nunca]
[Ocasionalmente]
[Siempre][Agendar cita médica] 

[Quiero aprovechar] [Más adelante] 

[Enviar pregunta]
[Labs/Recetas/Planes]
[Marcar a Clivi][Reenviar receta/labs]
[Reenviar orden labs]
[Enviar estudios][Reenviar receta - Última receta]
[Reenviar orden labs - Última orden de laboratorios]
[Reenviar estudios - Resultados de estudios]
[Reenviar plan nutrición - Último plan nutricional]


[Se resolvió] [Aún no] [Tengo otra duda difer

In [ ]:
menu_str = ""
for i, c in conv_frame_menus.iterrows():
  start_index = c["text"].find("[")
  end_index = len(c["text"])
  separated_menu = c["text"][start_index:end_index]
  menu_str = menu_str + separated_menu

In [ ]:
print(menu_str)

[Clivi] 

[Entrar a la sesión] 

[Preguntas y estudios]
[Citas]
[Mediciones][Entrar a entrenar] 

[Adelante - glucosa] 

[Confirmar] [Cancelar] 

[CLIVI - Peso] 

[Adelante - glucosa] 

[Ver recetario] 

[Entrar a entrenar] 

[Cancelar] 

[Adelante - glucosa] 

[Antes]
[Después][Ir a cuestionario] 

[Entrar a la cita] 

[No asistí a la cita] [Calificar cita] 

[Psicológica]
[Nutricional]
[Endocrinología][🙂🙂🙂]
[😐😐😐]
[🙁🙁🙁][Entrar a la sesión] 

[Entrar a entrenar] 

[Antes]
[Después][Adelante - glucosa] 

[Antes]
[Después][Entrar a la sesión] 

[Medidas] 

[Entrar a la sesión] 

[Entrar a entrenar] 

[Reagendar psicología] 

[Ver video] 

[Antes]
[Después][Adelante - glucosa] 

[Antes]
[Después][Adelante - glucosa] 

[Entrar a entrenar] 

[Adelante - glucosa] 

[Tratamiento CLIVI] 

[Entrar a la cita] 

[Requiero apoyo] [En camino] 

[Recibir mensaje] 

[Agendar ahora] 

[Entrar a la sesión] 

[Entrar a entrenar] 

[Ver video] 

[Adelante - glucosa] 

[Antes]
[Después][Entrar a sesión] 


In [ ]:
for i, c in conv_frame.iterrows():
  if c["text"].find("[") != -1:
    print(c["text"])


Tenemos nuevo asistente virtual en Clivi mejorado!

Hola! 💜

Necesitamos tu apoyo:

👉Guarda este número y elimina el anterior.
👉Pruébalo y escribe. Te damos unos ejemplos:
     *99 en ayunas*
     *Hola*
     *Quiero enviar una pregunta*

[Clivi] 


¡Hola! En unos momentos iniciaremos nuestro taller en vivo 5:00 p. m.. ⏰ 
La sesión será sobre "Cuidar - me hace querer - me ¿Qué es realmente tener salud?".

Tu asistencia es importante para cumplir con tu programa y tratamiento en CLIVI. ¡Te esperamos! 😊👩‍🏫

[Entrar a la sesión] 


¿Cómo te podemos ayudar?
[Preguntas y estudios]
[Citas]
[Mediciones]
¡Hola! esta es la liga para entrenar por si no te llegó el recordatorio.

Presiona el botón de abajo o escribe *Entrar a entrenar*

[Entrar a entrenar] 


Buenos días ☀️. Hoy debemos medirnos la glucosa antes de desayunar ☕. Por favor presiona “Adelante - glucosa” cuando estés list@ para enviar tu medición.

[Adelante - glucosa] 


¡Hola! ⏰ Es hora de confirmar tu cita de Nutrición el lunes 11

In [ ]:
conv_frame['createdOn'] = pd.to_datetime(conv_frame['createdOn'])


In [ ]:
conv_frame_sorted = conv_frame.sort_values(by='createdOn')


In [ ]:
history_conv= ""
for i,c in conv_frame_sorted.iterrows():
  strConv = "{}: {} : {} \n".format(c["userType"], c["text"], c["createdOn"])
  history_conv = history_conv + strConv

In [ ]:
print(history_conv)

BOT: Template: lg_subscribe_activate_msi_1 : 2023-08-10 20:06:58.684000+00:00 
BOT: Template: welcome_video_verify_whatsapp_number : 2023-08-10 20:15:34.856000+00:00 
USER: Iniciar con Clivi : 2023-08-10 20:16:00.833000+00:00 
USER: Agendar bienvenida : 2023-08-10 20:16:11.553000+00:00 
BOT: Template: appointment_book_calendly : 2023-08-10 20:16:12.162000+00:00 
BOT: Template: after_booking_onboarding_px : 2023-08-10 20:17:37.400000+00:00 
BOT: Template: reminder_appointment_confirmation_c : 2023-08-12 17:40:21.865000+00:00 
BOT: Template: reminder_appointment_confirmation_c : 2023-08-12 17:43:00.761000+00:00 
BOT: Template: reminder_appointment_confirmation_c : 2023-08-12 17:44:36.214000+00:00 
BOT: Template: reminder_appointment_confirmation_c : 2023-08-12 17:46:05.456000+00:00 
USER: Consultas : 2023-08-12 17:47:22.342000+00:00 
BOT: 🧠 *Cita de Psicología*
       lunes 14/8 4:00 p. m. - 5:00
       https://calendly.com/events/cf298baf-00a1-4a0f-9a72-684f9da62e62/google_meet

🥼 *Cita

In [ ]:
question_frame = getFullCollection('questions')

In [ ]:
question_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143130 entries, 0 to 143129
Data columns (total 50 columns):
 #   Column                        Non-Null Count   Dtype              
---  ------                        --------------   -----              
 0   modelQuestionId               128298 non-null  object             
 1   completedAt                   98443 non-null   datetime64[ns, UTC]
 2   isVip                         137837 non-null  object             
 3   operatorIdCompleted           135909 non-null  object             
 4   timeCompleted                 105706 non-null  datetime64[ns, UTC]
 5   whatsapp                      142525 non-null  object             
 6   msg                           142603 non-null  object             
 7   operatorName                  105247 non-null  object             
 8   patientBusinessId             142557 non-null  object             
 9   name                          142603 non-null  object             
 10  status              

In [ ]:
question_frame[question_frame["docId"] == "WMynM96hwguxlCDPb6SM"][["whatsapp", "msg"]]

,whatsapp,msg
74808,5216631001086,estoy en espera


In [ ]:
question_frame["timeSubmitted"] =  pd.to_datetime(question_frame["timeSubmitted"])

In [ ]:
fecha_deseada = '2024-06-26'
resultados_por_dia = question_frame[question_frame['timeSubmitted'].dt.date == pd.to_datetime(fecha_deseada).date()]

In [ ]:
resultados_por_dia

,userId,status,patientBusinessId,completedAt,timeAssigned,modelQuestionId,timeCompleted,whatsapp,operatorName,operatorIdAssigned,...,reassignedAt,dateCompleted,completedById,patientTreatments,sentiment,operatorId,isDuplicate,compeletedById,assignedByOperatorId,userType
884,5215551446398,COMPLETE,0,NaT,NaT,5ce1dc0a-515b-44b0-8cb8-4221f9d02b45,NaT,5215551446398,NaN,None,...,NaT,NaT,NaN,[DIABETES],"{'score': -0.09625238925218582, 'magnitude': 0...",NaN,NaN,NaN,NaN,NaN
948,5219931050929,COMPLETE,0,NaT,NaT,578bfb89-6466-4a6a-a345-84235434c0ab,2024-06-26 13:45:18.229000+00:00,5219931050929,NaN,None,...,NaT,NaT,NaN,[DIABETES],"{'score': 0.014376657083630562, 'magnitude': 0...",NaN,NaN,NaN,NaN,NaN
1115,5215539795882,COMPLETE,0,NaT,NaT,b547bd7e-3d7a-4691-8be8-288972c4a916,2024-06-26 14:09:16.718000+00:00,5215539795882,Andrea Balderas,None,...,NaT,NaT,NaN,[OBESITY],None,NaN,NaN,NaN,NaN,NaN
1212,5218123543043,COMPLETE,0,NaT,NaT,4513a131-bf15-4cc1-a87c-2fd30a109e3e,2024-06-26 17:57:18.783000+00:00,5218123543043,Andrea Balderas,None,...,NaT,NaT,NaN,[OBESITY],"{'score': 0.06438405066728592, 'magnitude': 0....",NaN,NaN,NaN,NaN,NaN
1351,5218118609039,COMPLETE,0,NaT,NaT,18d5a0d0-425c-4b49-9389-6b030b22a93b,NaT,5218118609039,NaN,None,...,NaT,NaT,NaN,[OBESITY],"{'magnitude': 0.14881962537765503, 'score': -0...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138961,5215585564994,COMPLETE,0,NaT,NaT,b13367ab-e2f1-4aa5-8bb8-b5081c237c9a,2024-06-26 17:01:04.671000+00:00,5215585564994,Andrea Balderas,None,...,NaT,NaT,NaN,[OBESITY],"{'score': -0.027221299707889557, 'magnitude': ...",NaN,NaN,NaN,NaN,NaN
139663,5215631699336,COMPLETE,0,NaT,NaT,8cfe86c0-4dee-443c-87f8-0d194ad9ad67,2024-06-26 15:56:45.812000+00:00,5215631699336,Andrea Balderas,None,...,NaT,NaT,NaN,[OBESITY],"{'magnitude': 0.5367754697799683, 'score': -0....",NaN,NaN,NaN,NaN,NaN
140384,5219988453702,COMPLETE,0,NaT,NaT,8d79b6ee-7c58-4dc4-b752-f574f3a4cf7a,2024-06-26 16:01:37.821000+00:00,5219988453702,NaN,None,...,NaT,NaT,NaN,[DIABETES],None,NaN,NaN,NaN,NaN,NaN
140731,5213310893557,COMPLETE,0,NaT,NaT,a4d7e629-52f7-4c7e-a7c5-40b60efe7e7d,2024-06-26 18:07:55.110000+00:00,5213311186891,Juan Carlos Zaragoza,None,...,NaT,NaT,NaN,[DIABETES],None,NaN,NaN,NaN,NaN,NaN


In [ ]:
resultados_por_dia["whatsapp"].value_counts().reset_index()

,whatsapp,count
0,5219611208927,5
1,5215631699336,5
2,5218140054111,4
3,5215554082704,4
4,5215570811826,4
...,...,...
191,5215548746603,1
192,5219142798084,1
193,5216311778583,1
194,5218116136387,1


### Convertir a un dataframe